In [1]:
module My

"""
    savevar(fn, x)

saves the value of `x` to the file `fn`, where `fn` is the filename string of the file.
"""
savevar(fn, x) = write(fn, string(x))

"""
    loadvar(fn)

loads the file `fn` (the filename string of the file) and `Meta.parse |> eval`.
"""
loadvar(fn) = read(fn, String) |> Meta.parse |> eval

"""
    dir_savevar[]

is the default directory to which `@savevar` saves the values of variables.
"""
const dir_savevar = Ref(".")

"""
    fn_savevar(x::Symbol)

is the filename string to which `@savevar` saves the value of a variable.
"""
fn_savevar(x::Symbol) = joinpath(dir_savevar[], string(x) * ".txt")

"""
    @savevar(args...)

saves the variables in args to the corresponding textfiles.

Example: `@savevar A B C` saves the variables `A`, `B`, `C` to textfiles. 
"""
macro savevar(args...)
    A = [:(savevar($(fn_savevar(x)), $(esc(x)))) for x in args]
    quote $(A...); nothing end
end

"""
    @loadvar(args...)

loads the values from the textfiles corresponding to `args`.
If `length(args)` is greater than 1, then it returns the tuple of the values.

Example: `a, b, c = @loadvar A B C` loads 
the values of `A`, `B`, `C` in textfiles to the variables `a`, `b`, `c`.
"""
macro loadvar(args...)
    if length(args) == 1
        x = args[1]
        :(loadvar($(fn_savevar(x))))
    else
        A = [:(loadvar($(fn_savevar(x)))) for x in args]
        :(($(A...),))
    end
end

end

Main.My

In [2]:
using .My: dir_savevar, @savevar, @loadvar
dir_savevar[] = "tmp"

"tmp"

In [3]:
?dir_savevar

search:



```
dir_savevar[]
```

is the default directory to which `@savevar` saves the values of variables.


In [4]:
?@savevar

```
@savevar(args...)
```

saves the variables in args to the corresponding textfiles.

Example: `@savevar A B C` saves the variables `A`, `B`, `C` to textfiles. 


In [5]:
?@loadvar

```
@loadvar(args...)
```

loads the values from the textfiles corresponding to `args`. If `length(args)` is greater than 1, then it returns the tuple of the values.

Example: `a, b, c = @loadvar A B C` loads  the values of `A`, `B`, `C` in textfiles to the variables `a`, `b`, `c`.


In [6]:
using Random; Random.seed!(4649373)

MersenneTwister(4649373)

In [7]:
A = randn(ComplexF64, 4, 3, 2)

4×3×2 Array{ComplexF64, 3}:
[:, :, 1] =
  0.357345-1.2478im     0.682675-0.19939im    -0.828604-0.130312im
  0.999778-0.128819im   0.356639+0.0984794im  -0.249303+0.0792775im
  0.646351+0.287488im   -1.63661-1.1631im     0.0147973-0.0963827im
 -0.838917+0.305942im  -0.276665-0.339579im     1.02587-0.00997503im

[:, :, 2] =
 -0.0701421+1.07701im     0.229101+0.35718im    -1.06954+0.966731im
   0.212719+0.0450435im  -0.212064-1.05741im   -0.100229+0.0276602im
    1.22601-0.671355im    0.507781+0.146455im   0.131755+1.1808im
    0.20559+0.687265im   -0.407012-1.16322im   -0.130588-0.832639im

In [8]:
@savevar A

In [9]:
read("tmp/A.txt", String)

"[0.3573446150049466 - 1.247796689524791im 0.6826751518535276 - 0.19938984132658544im -0.828604404251341 - 0.13031243958002936im; 0.9997775125877176 - 0.1288185084503161im 0.3566393536097865 + 0.09847938776643947im -0.24930302594101603 + 0.07927746587437946im; 0.646350739" ⋯ 499 bytes ⋯ "02270299698im; 1.226008377537175 - 0.6713548316258817im 0.5077812548164254 + 0.14645450926431483im 0.1317552128204387 + 1.1807955920537025im; 0.2055903575599693 + 0.6872649191653237im -0.4070115367381583 - 1.1632213017570572im -0.13058807817013474 - 0.832638913474766im]"

In [10]:
A_load = @loadvar A

4×3×2 Array{ComplexF64, 3}:
[:, :, 1] =
  0.357345-1.2478im     0.682675-0.19939im    -0.828604-0.130312im
  0.999778-0.128819im   0.356639+0.0984794im  -0.249303+0.0792775im
  0.646351+0.287488im   -1.63661-1.1631im     0.0147973-0.0963827im
 -0.838917+0.305942im  -0.276665-0.339579im     1.02587-0.00997503im

[:, :, 2] =
 -0.0701421+1.07701im     0.229101+0.35718im    -1.06954+0.966731im
   0.212719+0.0450435im  -0.212064-1.05741im   -0.100229+0.0276602im
    1.22601-0.671355im    0.507781+0.146455im   0.131755+1.1808im
    0.20559+0.687265im   -0.407012-1.16322im   -0.130588-0.832639im

In [11]:
A_load == A

true

In [12]:
B = ["Foo", "Bar", "Baz"]

3-element Vector{String}:
 "Foo"
 "Bar"
 "Baz"

In [13]:
@savevar B

In [14]:
read("tmp/B.txt", String)

"[\"Foo\", \"Bar\", \"Baz\"]"

In [15]:
B_load = @loadvar B

3-element Vector{String}:
 "Foo"
 "Bar"
 "Baz"

In [16]:
B_load == B

true

In [17]:
D = Dict(:A => A, :B => B)

Dict{Symbol, Array} with 2 entries:
  :A => [0.357345-1.2478im 0.682675-0.19939im -0.828604-0.130312im; 0.999778-0.…
  :B => ["Foo", "Bar", "Baz"]

In [18]:
@savevar D

In [19]:
read("tmp/D.txt", String)

"Dict{Symbol, Array}(:A => [0.3573446150049466 - 1.247796689524791im 0.6826751518535276 - 0.19938984132658544im -0.828604404251341 - 0.13031243958002936im; 0.9997775125877176 - 0.1288185084503161im 0.3566393536097865 + 0.09847938776643947im -0.24930302594101603 + 0.079277" ⋯ 555 bytes ⋯ "75 - 0.6713548316258817im 0.5077812548164254 + 0.14645450926431483im 0.1317552128204387 + 1.1807955920537025im; 0.2055903575599693 + 0.6872649191653237im -0.4070115367381583 - 1.1632213017570572im -0.13058807817013474 - 0.832638913474766im], :B => [\"Foo\", \"Bar\", \"Baz\"])"

In [20]:
D_load = @loadvar D

Dict{Symbol, Array} with 2 entries:
  :A => [0.357345-1.2478im 0.682675-0.19939im -0.828604-0.130312im; 0.999778-0.…
  :B => ["Foo", "Bar", "Baz"]

In [21]:
D_load == D

true

In [22]:
module O
struct Foo{A, B}
    a::A
    b::B
end
end

Main.O

In [23]:
foo = O.Foo(A, B)

Main.O.Foo{Array{ComplexF64, 3}, Vector{String}}([0.3573446150049466 - 1.247796689524791im 0.6826751518535276 - 0.19938984132658544im -0.828604404251341 - 0.13031243958002936im; 0.9997775125877176 - 0.1288185084503161im 0.3566393536097865 + 0.09847938776643947im -0.24930302594101603 + 0.07927746587437946im; 0.6463507394027657 + 0.2874875698910842im -1.6366131682001521 - 1.1630981589087876im 0.01479727941198744 - 0.09638269986747496im; -0.8389169497092693 + 0.3059416490070709im -0.276664821700539 - 0.33957890029521076im 1.0258672083835658 - 0.00997502769507162im;;; -0.07014210671299528 + 1.077013678925162im 0.22910085513407732 + 0.3571799374969964im -1.0695368505074354 + 0.9667307185166448im; 0.21271939342846552 + 0.0450434630802757im -0.2120637210443527 - 1.0574146943973246im -0.10022936815590866 + 0.027660202270299698im; 1.226008377537175 - 0.6713548316258817im 0.5077812548164254 + 0.14645450926431483im 0.1317552128204387 + 1.1807955920537025im; 0.2055903575599693 + 0.6872649191653237

In [24]:
@savevar foo

In [25]:
read("tmp/foo.txt", String)

"Main.O.Foo{Array{ComplexF64, 3}, Vector{String}}([0.3573446150049466 - 1.247796689524791im 0.6826751518535276 - 0.19938984132658544im -0.828604404251341 - 0.13031243958002936im; 0.9997775125877176 - 0.1288185084503161im 0.3566393536097865 + 0.09847938776643947im -0.24930" ⋯ 572 bytes ⋯ "77537175 - 0.6713548316258817im 0.5077812548164254 + 0.14645450926431483im 0.1317552128204387 + 1.1807955920537025im; 0.2055903575599693 + 0.6872649191653237im -0.4070115367381583 - 1.1632213017570572im -0.13058807817013474 - 0.832638913474766im], [\"Foo\", \"Bar\", \"Baz\"])"

In [26]:
foo_load = @loadvar foo

Main.O.Foo{Array{ComplexF64, 3}, Vector{String}}([0.3573446150049466 - 1.247796689524791im 0.6826751518535276 - 0.19938984132658544im -0.828604404251341 - 0.13031243958002936im; 0.9997775125877176 - 0.1288185084503161im 0.3566393536097865 + 0.09847938776643947im -0.24930302594101603 + 0.07927746587437946im; 0.6463507394027657 + 0.2874875698910842im -1.6366131682001521 - 1.1630981589087876im 0.01479727941198744 - 0.09638269986747496im; -0.8389169497092693 + 0.3059416490070709im -0.276664821700539 - 0.33957890029521076im 1.0258672083835658 - 0.00997502769507162im;;; -0.07014210671299528 + 1.077013678925162im 0.22910085513407732 + 0.3571799374969964im -1.0695368505074354 + 0.9667307185166448im; 0.21271939342846552 + 0.0450434630802757im -0.2120637210443527 - 1.0574146943973246im -0.10022936815590866 + 0.027660202270299698im; 1.226008377537175 - 0.6713548316258817im 0.5077812548164254 + 0.14645450926431483im 0.1317552128204387 + 1.1807955920537025im; 0.2055903575599693 + 0.6872649191653237

In [27]:
(foo_load.a, foo_load.b) == (foo.a, foo.b)

true

In [28]:
x, y, z = randn(3)
x, y, z

(0.25809857024548255, -0.6921471091207019, 0.3180664127334103)

In [29]:
@savevar x y z

In [30]:
X, Y, Z = @loadvar x y z
X, Y, Z

(0.25809857024548255, -0.6921471091207019, 0.3180664127334103)

In [31]:
(X, Y, Z) == (x, y, z)

true